In [1]:
%pip install polars
%pip install tiktoken
%pip install llama-index

Note: you may need to restart the kernel to use updated packages.


'c:\Users\Leonce' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.


'c:\Users\Leonce' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.


'c:\Users\Leonce' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import os
import getpass
import pandas as pd 
import polars as pl 
import numpy as np
import os
import re
import sys
import time

from polars import col, lit, when

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

c:\Users\Leonce Nshuti\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Leonce Nshuti\miniconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\Leonce Nshuti\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [4]:
df = pd.read_csv('data/Metro_zori_sm_month.csv')
df = pl.from_pandas(df)
df.head(5)

RegionID,SizeRank,RegionName,RegionType,StateName,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,...,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30
i64,i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
102001,0,"""United States""","""country""",null,1316.882137,1327.077746,1336.985344,1347.189723,1353.394167,1359.074745,1361.02959,1362.484521,1361.99959,1361.649193,1364.550876,1369.963599,1379.309944,1386.898118,1395.134067,1401.469517,1406.765645,1409.035038,1408.899558,1407.306618,1405.202237,1404.132406,1407.010103,1414.139039,1423.957912,1434.320507,1444.260427,1452.980349,1459.675943,1462.720983,1462.758291,1460.658616,...,1594.986052,1590.984243,1588.015929,1593.384711,1595.956018,1594.258842,1590.659951,1590.818415,1593.232693,1598.663102,1606.403315,1622.052751,1645.434281,1676.624721,1711.37393,1748.556406,1784.305473,1812.840551,1830.959961,1842.632765,1853.709904,1862.312776,1878.041155,1893.431256,1917.160444,1939.707277,1965.396128,1987.4913,2002.482563,2008.140297,2006.081681,1997.638718,1992.421079,1991.266756,1997.22861,2006.354216,2018.055353
394913,1,"""New York, NY""","""msa""","""NY""",2443.764214,2464.71937,2483.19687,2500.971887,2512.71835,2527.235246,2535.637527,2538.121825,2530.41857,2520.939911,2514.393078,2517.074501,2522.602645,2533.320729,2545.975934,2562.720678,2575.710545,2579.940612,2580.827517,2572.804192,2565.740194,2552.94805,2545.638928,2545.900785,2553.484774,2568.132066,2586.419243,2604.540608,2620.435513,2625.28689,2626.368054,2615.597126,...,2734.003853,2714.528617,2687.140254,2660.915081,null,null,2512.370804,2480.089537,2464.501276,2462.847972,2466.083846,2485.388868,2516.052248,2565.91162,2622.574179,2687.832686,2749.730245,2802.92839,2836.559578,2858.662482,2876.972047,2898.104052,2935.461154,2978.231449,3030.895777,3089.224194,3151.866917,3207.49192,3233.573512,3231.414682,3207.885431,3175.252087,3151.638239,3143.696116,3159.468131,3188.554355,3229.346478
753899,2,"""Los Angeles, C...","""msa""","""CA""",1921.951938,1935.567889,1948.902894,1961.209437,1978.339689,1992.539805,2003.693107,2007.515047,2011.865137,2016.321057,2025.271124,2038.785648,2053.023696,2064.79274,2077.040621,2089.577254,2103.506829,2110.822049,2118.259875,2120.41636,2122.129423,2123.588364,2133.503842,2148.927275,2166.645502,2182.24086,2195.391682,2206.189849,2216.876899,2225.034927,2232.525823,2235.515281,...,null,null,2387.710691,2389.404037,2389.376732,2390.773149,2391.319141,2398.43669,2399.822633,2402.104678,2406.076988,2419.904232,2439.078322,2467.206528,2504.650324,2554.244345,2613.455887,2660.652851,2695.543962,2713.159599,2733.626302,2750.565013,2775.179103,2802.062323,2835.175022,2867.878174,2893.926879,2916.787977,2931.511579,2939.060199,2934.669745,2926.508092,2918.427897,2913.35203,2916.907462,2926.201136,2940.362187
394463,3,"""Chicago, IL""","""msa""","""IL""",1490.453763,1502.13515,1513.653098,1523.141742,1529.525405,1533.51917,1532.799133,1526.916057,1517.062534,1513.560152,1515.842077,1526.465707,1536.860364,1547.478507,1556.899427,1565.417534,1573.15787,1576.370639,1570.27137,1558.180442,1545.853754,

In [5]:
df = pd.read_csv('data/nmdb-new-mortgage-statistics-national-census-areas-quarterly.csv')
df = pl.from_pandas(df)
df.head(5)

SOURCE,FREQUENCY,SERIESID,GEOLEVEL,GEOID,GEONAME,MARKET,PERIOD,YEAR,QUARTER,MONTH,SUPPRESSED,VALUE1
str,str,str,str,str,str,str,str,i64,i64,i64,i64,f64
"""NMDB""","""Quarterly""","""TOT_ORIG""","""National""","""USA""","""United States""","""All Mortgages""","""1998Q1""",1998,1,3,0,2549.0
"""NMDB""","""Quarterly""","""TOT_ORIG""","""National""","""USA""","""United States""","""All Mortgages ...","""1998Q1""",1998,1,3,0,1046.0
"""NMDB""","""Quarterly""","""TOT_ORIG""","""National""","""USA""","""United States""","""All Mortgages ...","""1998Q1""",1998,1,3,0,1503.0
"""NMDB""","""Quarterly""","""TOT_ORIG""","""National""","""USA""","""United States""","""All Mortgages""","""1998Q2""",1998,2,6,0,2916.0
"""NMDB""","""Quarterly""","""TOT_ORIG""","""National""","""USA""","""United States""","""All Mortgages ...","""1998Q2""",1998,2,6,0,1486.0


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
llm = OpenAI(temperature=0)

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
from langchain.document_loaders import TextLoader
loader = CSVLoader(file_path='data/Metro_zori_sm_month.csv')

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


# Keep 100 elements from texts 
texts = texts[:5]

embeddings = OpenAIEmbeddings()
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [ ]:
texts

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="a dataframe",
    vectorstore=state_of_union_store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("How many rows in dataframe?")